In [4]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from genai.client import Client
from genai.credentials import Credentials
from genai.schema import TextGenerationParameters, TextGenerationReturnOptions
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationHAPOutput,
    ModerationParameters,
    TextGenerationParameters,
)
from IPython.display import display, Code
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:3000,
    GenParams.MIN_NEW_TOKENS:1,
    # GenParams.TEMPERATURE:0.5,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```"],
}

# model = Model("meta-llama/llama-2-70b-chat",creds, params, project_id)
# model = Model("codellama/codellama-34b-instruct-hf",creds, params, project_id)

model = LangChainInterface(
        model_id="ibm/granite-20b-code-instruct-v1",
        # mistralai/mixtral-8x7b-instruct-v0-1",
        client=Client(credentials=Credentials.from_env()),
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.GREEDY,
            max_new_tokens=3000,
            min_new_tokens=1,
            # temperature=0.05,
            top_k=50,
            top_p=1,
            stop_sequences=['```'],
        )
    )

In [5]:
def keep_generate_till(model,prompt,endstring):
    lastanswer = ""
    while not lastanswer.endswith(endstring):
        answer = model.invoke(f"{prompt}{lastanswer}")
        lastanswer += answer
        # print(answer)
    return lastanswer.replace('```','')

In [3]:
from IPython.display import display, Code

java = """
import java.math.BigDecimal;
import java.math.BigInteger;
import java.math.RoundingMode;
public class InterestNoRoundingConst {
    public static void main(String[] args) {
        BigDecimal b = new BigDecimal(BigInteger.valueOf(963807195502L), 2);
        BigDecimal c = new BigDecimal(BigInteger.valueOf(180945931154L), 2);
        BigDecimal p = new BigDecimal(BigInteger.valueOf(81090), 6);
        short n = 30;
        // COMPUTE MN = (1 + P) ** N
        BigDecimal mn = BigDecimal.ONE.add(p).pow(n);
        // COMPUTE B = (B + C) * MN + C * ((MN - 1) / P - 1).
        b = b.add(c).multiply(mn).add(
        c.multiply(mn.subtract(BigDecimal.ONE).divide(p, RoundingMode.DOWN).subtract(BigDecimal.ONE)));
        System.out.printf("$%17.2f\n", b);
        System.out.println(new DecimalFormat("$#,##0.##").format(b));
    }
}
"""

bestpratice = """
    import com.ibm.arithmetic.decimal.DecimalPrecision;
    import com.ibm.arithmetic.decimal.DecimalVariable;
    import com.ibm.arithmetic.decimal.impl.daa.DAAPrecision;
    import com.ibm.arithmetic.decimal.impl.daa.DAAFactory;
    import com.ibm.arithmetic.decimal.impl.daa.DAADecimal;
    public class foo {
        private static final DAAFactory df = new DAAFactory();
        public static void bar(){
        DecimalPrecision<DAADecimal> v1 = new DecimalPrecision<>(df, 17, 2, false);
        DecimalVariable<DAADecimal> v2 = df.createDecimalVariable(v1, df.createDecimal(12345, 2));
        DAADecimal v3 = df.createDecimal(12345, 2);
    }
}"""

prompt = f"""[INST]<<SYS>>
java: `{java}`
best practice: `{bestpratice}`
<</SYS>>
as expert on java, please tune up the performance of the java code provided.
-generate the java only.
-use com.ibm.arithmetic.decimal.impl.daa.DAADecimal, com.ibm.arithmetic.decimal.impl.daa.DAAFactory, com.ibm.arithmetic.decimal.impl.daa.DAAPrecision com.ibm.arithmetic.decimal.DecimalPrecision, com.ibm.arithmetic.decimal.DecimalVariable if possible.
-employ the best practice learned.
-append a comment of the line you updated that describe your work.
-dont show the notes.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
as expert on java, please tune up the performance of the java code provided.\
-generate the java only.\
-use com.ibm.arithmetic.decimal.impl.daa.DAADecimal, com.ibm.arithmetic.decimal.impl.daa.DAAFactory, com.ibm.arithmetic.decimal.impl.daa.DAAPrecision com.ibm.arithmetic.decimal.DecimalPrecision, com.ibm.arithmetic.decimal.DecimalVariable if possible.\
-employ the best practice learned.\
-append a comment of the line you updated that describe your work.\
-dont show the notes.\
-end with ```.
java: `{java}`
best practice: `{bestpratice}`
Answer:```java"""

display(Code(java,language="java"))
answer = model.invoke(prompt)
display(Code(answer,language="java"))

import java.math.BigDecimal;
import java.math.BigInteger;
import java.math.RoundingMode;
public class InterestNoRoundingConst {
    public static void main(String[] args) {
        BigDecimal b = new BigDecimal(BigInteger.valueOf(963807195502L), 2);
        BigDecimal c = new BigDecimal(BigInteger.valueOf(180945931154L), 2);
        BigDecimal p = new BigDecimal(BigInteger.valueOf(81090), 6);
        short n = 30;
        // COMPUTE MN = (1 + P) ** N
        BigDecimal mn = BigDecimal.ONE.add(p).pow(n);
        // COMPUTE B = (B + C) * MN + C * ((MN - 1) / P - 1).
        b = b.add(c).multiply(mn).add(
        c.multiply(mn.subtract(BigDecimal.ONE).divide(p, RoundingMode.DOWN).subtract(BigDecimal.ONE)));
        System.out.printf("$%17.2f
", b);
        System.out.println(new DecimalFormat("$#,##0.##").format(b));
    }
}

import com.ibm.arithmetic.decimal.DecimalPrecision;
import com.ibm.arithmetic.decimal.DecimalVariable;
import com.ibm.arithmetic.decimal.impl.daa.DAAPrecision;
import com.ibm.arithmetic.decimal.impl.daa.DAAFactory;
import com.ibm.arithmetic.decimal.impl.daa.DAADecimal;
public class InterestNoRoundingConst {
    public static void main(String[] args) {
        // COMPUTE B = (B + C) * MN + C * ((MN - 1) / P - 1).
        DAADecimal b = new DAADecimal(BigInteger.valueOf(963807195502L), 2);
        DAADecimal c = new DAADecimal(BigInteger.valueOf(180945931154L), 2);
        DAADecimal p = new DAADecimal(BigInteger.valueOf(81090), 6);
        short n = 30;
        // COMPUTE MN = (1 + P) ** N
        DAADecimal mn = DAADecimal.ONE.add(p).pow(n);
        b = b.add(c).multiply(mn).add(
        c.multiply(mn.subtract(DAADecimal.ONE).divide(p, RoundingMode.DOWN).subtract(DAADecimal.ONE)));
        System.out.printf("$%17.2f
", b);
        System.out.println(new DecimalFormat("$#,##0.##").format(b));
    }
}
```

In [19]:
#Not user friendly to developer	
#comments inside COBOL had better move to JAVA to make it more readable to java developer. 
#[RFE#WCAST-I-53]

cobol = """
       C200-CONV-CIFEPI-ACCT-RTN.


      *        *** CALCULATE THE NUMBER OF CHEQUES RETURNED
           MOVE ZERO TO  WS-300
           PERFORM UNTIL WS-300 = 300
               IF CCINF-EXCESS-DATE NOT = X'000000'
                   ADD NUM-1         TO WS-IF2-CNT
                   IF CCINF-EXCESS-DATE = WS-SYS-DATE-X3
                       ADD NUM-1         TO WS-IF3-CNT

                       MOVE CCINF-CHQ-RETURN-THIS-PRD TO WS-TEMP-X1
                       ADD NUM-1 TO WS-TEMP-HEX-TO-DCML2
                       ADD NUM-1         TO WS-ADD1-CNT

                   ELSE
                       ADD NUM-1         TO WS-IF3-CNT

                       MOVE CCINF-CHQ-RETURN-LAST-PRD TO WS-TEMP-X3
                       ADD NUM-1 TO WS-TEMP-HEX-TO-DCML3
                       ADD NUM-1         TO WS-ADD1-CNT

                   END-IF
               END-IF
               ADD NUM-1 TO      WS-300
           END-PERFORM
           MOVE WS-TEMP-X1 TO CCINF-CHQ-RETURN-THIS-PRD
           MOVE WS-TEMP-X3 TO CCINF-CHQ-RETURN-LAST-PRD
"""

java = """
    public void c200ConvCifepiAcctRtn() {
        CntWorkArea cntWorkArea = new CntWorkArea();
        ConstWorkArea constWorkArea = new ConstWorkArea();
        Filler13 filler13 = new Filler13();
        SysDateTimeWorkArea sysDateTimeWorkArea = new SysDateTimeWorkArea();
        BigDecimal wsccinfinstlamtn = BigDecimal.ZERO;


        cntWorkArea.setWs300(0);
        while (cntWorkArea.getWs300() < 300) {
            if (!"000000".equals(this.ccinfExcessDate)) {
                cntWorkArea.setWsIf2Cnt(cntWorkArea.getWsIf2Cnt() + constWorkArea.getNum1());
                if (this.ccinfExcessDate.equals(sysDateTimeWorkArea.getWsSysDateX3())) {
                    cntWorkArea.setWsIf3Cnt(cntWorkArea.getWsIf3Cnt() + constWorkArea.getNum1());
                    filler13.setWsTempX1(this.ccinfChqReturnThisPrd);
                    filler13.setWsTempHexToDcml2(filler13.getWsTempHexToDcml2() + constWorkArea.getNum1());
                    cntWorkArea.setWsAdd1Cnt(cntWorkArea.getWsAdd1Cnt() + constWorkArea.getNum1());
                } else {
                    cntWorkArea.setWsIf3Cnt(cntWorkArea.getWsIf3Cnt() + constWorkArea.getNum1());
                    filler13.setWsTempX3(this.ccinfChqReturnLastPrd);
                    filler13.setWsTempHexToDcml3(filler13.getWsTempHexToDcml3() + constWorkArea.getNum1());
                    cntWorkArea.setWsAdd1Cnt(cntWorkArea.getWsAdd1Cnt() + constWorkArea.getNum1());
                }
            }
            System.out.print("-"+cntWorkArea.getWs300());
            cntWorkArea.setWs300(cntWorkArea.getWs300() + constWorkArea.getNum1());
        }
"""

In [21]:
prompt = f"""[INST]<<SYS>>
cobol: `{cobol}`
java: `{java}`
<</SYS>>
please insert the comment extracted from COBOL code to the JAVA code.
-generate the java only.
-dont show the notes.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
you are expert of both COBOL and java.\
please copy the comment from COBOL code to the JAVA code.\
-extract comment from COBOL\
-add detail comment to original Java code\
-generate the java only.\
-dont show the notes.\
COBOL: `{cobol}`
original java: `{java}`
Answer:```java with detail comment"""

model = LangChainInterface(
        model_id="ibm/granite-20b-code-instruct-v1",
        client=Client(credentials=Credentials.from_env()),
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.GREEDY,
            max_new_tokens=3000,
            min_new_tokens=1,
            top_k=50,
            top_p=1,
            stop_sequences=['```'],
        )
    )
# answer = model.invoke(prompt)
answer = keep_generate_till(model,prompt,'```')

display(Code(cobol,language="cobol"))
display(Code(answer,language="java"))

C200-CONV-CIFEPI-ACCT-RTN.


      *        *** CALCULATE THE NUMBER OF CHEQUES RETURNED
           MOVE ZERO TO  WS-300
           PERFORM UNTIL WS-300 = 300
               IF CCINF-EXCESS-DATE NOT = X'000000'
                   ADD NUM-1         TO WS-IF2-CNT
                   IF CCINF-EXCESS-DATE = WS-SYS-DATE-X3
                       ADD NUM-1         TO WS-IF3-CNT

                       MOVE CCINF-CHQ-RETURN-THIS-PRD TO WS-TEMP-X1
                       ADD NUM-1 TO WS-TEMP-HEX-TO-DCML2
                       ADD NUM-1         TO WS-ADD1-CNT

                   ELSE
                       ADD NUM-1         TO WS-IF3-CNT

                       MOVE CCINF-CHQ-RETURN-LAST-PRD TO WS-TEMP-X3
                       ADD NUM-1 TO WS-TEMP-HEX-TO-DCML3
                       ADD NUM-1         TO WS-ADD1-CNT

                   END-IF
               END-IF
               ADD NUM-1 TO      WS-300
           END-PERFORM
           MOVE WS-TEMP-X1 TO CCINF-CHQ-RETURN-THIS-PRD
           MOVE WS-TEMP-X3 TO CCINF-CHQ-RETURN-LAST-PRD

/**
     * Calculate the number of cheques returned
     */
    public void c200ConvCifepiAcctRtn() {
        CntWorkArea cntWorkArea = new CntWorkArea();
        ConstWorkArea constWorkArea = new ConstWorkArea();
        Filler13 filler13 = new Filler13();
        SysDateTimeWorkArea sysDateTimeWorkArea = new SysDateTimeWorkArea();
        BigDecimal wsccinfinstlamtn = BigDecimal.ZERO;


        cntWorkArea.setWs300(0);
        while (cntWorkArea.getWs300() < 300) {
            if (!"000000".equals(this.ccinfExcessDate)) {
                cntWorkArea.setWsIf2Cnt(cntWorkArea.getWsIf2Cnt() + constWorkArea.getNum1());
                if (this.ccinfExcessDate.equals(sysDateTimeWorkArea.getWsSysDateX3())) {
                    cntWorkArea.setWsIf3Cnt(cntWorkArea.getWsIf3Cnt() + constWorkArea.getNum1());
                    filler13.setWsTempX1(this.ccinfChqReturnThisPrd);
                    filler13.setWsTempHexToDcml2(filler13.getWsTempHexToDcml2() + constWorkArea.getNum1());
                    cntWorkArea.setWsAdd1Cnt(cntWorkArea.getWsAdd1Cnt() + constWorkArea.getNum1());
                } else {
                    cntWorkArea.setWsIf3Cnt(cntWorkArea.getWsIf3Cnt() + constWorkArea.getNum1());
                    filler13.setWsTempX3(this.ccinfChqReturnLastPrd);
                    filler13.setWsTempHexToDcml3(filler13.getWsTempHexToDcml3() + constWorkArea.getNum1());
                    cntWorkArea.setWsAdd1Cnt(cntWorkArea.getWsAdd1Cnt() + constWorkArea.getNum1());
                }
            }
            cntWorkArea.setWs300(cntWorkArea.getWs300() + constWorkArea.getNum1());
        }
        this.ccinfChqReturnThisPrd = filler13.getWsTempX1();
        this.ccinfChqReturnLastPrd = filler13.getWsTempX3();
    }

In [22]:
#Not good java style
#it's responsiblity for JAVA class constructor to initialize variables  rather than setter method

cobol = """
      B010-INITIALIZATION SECTION.
      ******************************************************************
      * THIS SECTION IS USED FOR INITIALIZING THE PROGRAM              *
      ******************************************************************
       B010-10-START.
      *
           MOVE 'B010-INITIALIZATION'  TO WE-PARAGRAPH-NAME
      *
      **** INITIALIZE THE WORKING STORAGE VARIABLES AND FLAG VARIABLES
      *
           MOVE SPACES                 TO WW-VARIABLES
                                          WF-FLAG
           INITIALIZE WW-VARIABLES     REPLACING NUMERIC BY ZEROES
      *
      **** INITIALIZE COMP TYPE FIELDS
      *
           MOVE ZEROES                 TO WA-REC-READ
"""

java = """
    public static void b010Initialization(long waRecRead, WwVariables wwVariables) {
    WeErrorVariables weErrorVariables = new WeErrorVariables();

    weErrorVariables.setWeParagraphName(""B010-INITIALIZATION"");

    wwVariables.setWwVariables(""                "");
    wwVariables.setWfFlag(' ');
    wwVariables.setWwVariables(""0"".repeat(20));
    waRecRead = 0;
"""

In [23]:
prompt = f"""[INST]<<SYS>>
cobol: `{cobol}`
java: `{java}`
<</SYS>>
please rewrite JAVA code base on COBOL code.
-dont use the setter method.
-initialize variable with providing parameters in constructors.
-generate the java only.
-dont show the notes.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
please rewrite JAVA code base on COBOL code.\
-dont use the setter method.\
-initialize variable with providing parameters in constructors.\
-generate the java only.\
-dont show the notes.\
-end with ```.
cobol: `{cobol}`
java: `{java}`
Answer:```java"""

answer = model.invoke(prompt)
display(Code(answer,language="java"))

public static void b010Initialization(long waRecRead, WwVariables wwVariables) {
        WeErrorVariables weErrorVariables = new WeErrorVariables();

        weErrorVariables.setWeParagraphName("B010-INITIALIZATION");

        wwVariables.setWwVariables("                ");
        wwVariables.setWfFlag(' ');
        wwVariables.setWwVariables("0".repeat(20));
        waRecRead = 0;
    }
```

In [24]:
#Not good java style
#many unused methods generated inside java class looks like rubbish, hard to maintain. need to clean, 

cobol = """
"""

java = """
package com.ibm.wcaz.implementation;

import com.ibm.jzos.fields.ByteArrayField;
import com.ibm.jzos.fields.CobolDatatypeFactory;
import com.ibm.jzos.fields.StringField;
import com.ibm.wcaz.datamodel.Du0u0BankCntlParm;
import java.io.UnsupportedEncodingException;
import java.util.Arrays;
+A1public class U0idis1BankCntlParm implements Comparable<U0idis1BankCntlParm> {
    private String u0idis1SysUpdTs = "";
    private Du0u0BankCntlParm du0u0BankCntlParm = new Du0u0BankCntlParm();
    
    public U0idis1BankCntlParm() {}
    
    public U0idis1BankCntlParm(String u0idis1SysUpdTs, Du0u0BankCntlParm du0u0BankCntlParm) {
        this.u0idis1SysUpdTs = u0idis1SysUpdTs;
        this.du0u0BankCntlParm = du0u0BankCntlParm;
    }
    
    public U0idis1BankCntlParm(U0idis1BankCntlParm that) {
        this.u0idis1SysUpdTs = that.u0idis1SysUpdTs;
        this.du0u0BankCntlParm = new Du0u0BankCntlParm(that.du0u0BankCntlParm);
    }
    
    protected U0idis1BankCntlParm(byte[] bytes, int offset) {
        setBytes(bytes, offset);
    }
    
    protected U0idis1BankCntlParm(byte[] bytes) {
        this(bytes, 0);
    }
    
    public static U0idis1BankCntlParm fromBytes(byte[] bytes, int offset) {
        return new U0idis1BankCntlParm(bytes, offset);
    }
    
    public static U0idis1BankCntlParm fromBytes(byte[] bytes) {
        return fromBytes(bytes, 0);
    }
    
    public static U0idis1BankCntlParm fromBytes(String bytes) {
        try {
            return fromBytes(bytes.getBytes(factory.getStringEncoding()));
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }
    
    public String getU0idis1SysUpdTs() {
        return this.u0idis1SysUpdTs;
    }
    
    public void setU0idis1SysUpdTs(String u0idis1SysUpdTs) {
        this.u0idis1SysUpdTs = u0idis1SysUpdTs;
    }
    
    public Du0u0BankCntlParm getDu0u0BankCntlParm() {
        return this.du0u0BankCntlParm;
    }
    
    public void setDu0u0BankCntlParm(Du0u0BankCntlParm du0u0BankCntlParm) {
        this.du0u0BankCntlParm = du0u0BankCntlParm;
    }
    
    
    public String toString() {
        StringBuilder s = new StringBuilder();
        s.append("{ u0idis1SysUpdTs=\"");
        s.append(getU0idis1SysUpdTs());
        s.append("\"");
        s.append(", du0u0BankCntlParm=\"");
        s.append(getDu0u0BankCntlParm());
        s.append("\"");
        s.append("}");
        return s.toString();
    }
    
    public boolean equals(U0idis1BankCntlParm that) {
        return this.u0idis1SysUpdTs.equals(that.u0idis1SysUpdTs) &&
            this.du0u0BankCntlParm.equals(that.du0u0BankCntlParm);
    }
    
    @Override
    public boolean equals(Object that) {
        return (that instanceof U0idis1BankCntlParm) && this.equals((U0idis1BankCntlParm)that);
    }
    
    @Override
    public int hashCode() {
        return u0idis1SysUpdTs.hashCode() ^
            Integer.rotateLeft(du0u0BankCntlParm.hashCode(), 1);
    }
    
    @Override
    public int compareTo(U0idis1BankCntlParm that) {
        int c = 0;
        c = this.u0idis1SysUpdTs.compareTo(that.u0idis1SysUpdTs);
        if ( c != 0 ) return c;
        c = this.du0u0BankCntlParm.compareTo(that.du0u0BankCntlParm);
        return c;
    }
    
    // Start of COBOL-compatible binary serialization metadata
    private static CobolDatatypeFactory factory = new CobolDatatypeFactory();
    static {
        factory.setStringTrimDefault(true);
        factory.setStringEncoding("IBM-1047");
    }
    
    private static final StringField U0IDIS1SYSUPDTS = factory.getStringField(26);
    private static final ByteArrayField DU0U0BANKCNTLPARM = factory.getByteArrayField(Du0u0BankCntlParm.SIZE);
    public static final int SIZE = factory.getOffset();
    // End of COBOL-compatible binary serialization metadata
    
    public byte[] getBytes(byte[] bytes, int offset) {
        U0IDIS1SYSUPDTS.putString(u0idis1SysUpdTs, bytes, offset);
        du0u0BankCntlParm.getBytes(bytes, DU0U0BANKCNTLPARM.getOffset() + offset);
        return bytes;
    }
    
    public final byte[] getBytes(byte[] bytes) {
        return getBytes(bytes, 0);
    }
    
    public final byte[] getBytes() {
        return getBytes(new byte[numBytes()]);
    }
    
    public final String toByteString() {
        try {
            return new String(getBytes(), factory.getStringEncoding());
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }
    
    public void setBytes(byte[] bytes, int offset) {
        if (bytes.length < SIZE + offset) {
            byte[] newBytes = Arrays.copyOf(bytes, SIZE + offset);
            Arrays.fill(newBytes, bytes.length, SIZE + offset, (byte)0x40 /*default EBCDIC space character*/);
            bytes = newBytes;
        }
        u0idis1SysUpdTs = U0IDIS1SYSUPDTS.getString(bytes, offset);
        du0u0BankCntlParm.setBytes(bytes, DU0U0BANKCNTLPARM.getOffset() + offset);
    }
    
    
    public final void setBytes(byte[] bytes) {
        setBytes(bytes, 0);
    }
    
    public final void setBytes(String bytes) {
        try {
            setBytes(bytes.getBytes(factory.getStringEncoding()));
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }
    
    public int numBytes() {
        return SIZE;
    }
}
"""

In [25]:
prompt = f"""[INST]<<SYS>>
java: `{java}`
<</SYS>>
please rewrite JAVA code.
-remove the getter and setter code and unused methods.
-generate the java only.
-dont show the notes.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
please rewrite JAVA code.\
-remove the getter and setter code and unused methods.\
-generate the java only.\
-dont show the notes.\
-end with ```.
java: `{java}`
Answer:```java"""

answer = model.invoke(prompt)
display(Code(answer,language="java"))

package com.ibm.wcaz.implementation;

import com.ibm.wcaz.datamodel.Du0u0BankCntlParm;

public class U0idis1BankCntlParm {
    private String u0idis1SysUpdTs = "";
    private Du0u0BankCntlParm du0u0BankCntlParm = new Du0u0BankCntlParm();
    
    public U0idis1BankCntlParm() {}
    
    public U0idis1BankCntlParm(String u0idis1SysUpdTs, Du0u0BankCntlParm du0u0BankCntlParm) {
        this.u0idis1SysUpdTs = u0idis1SysUpdTs;
        this.du0u0BankCntlParm = du0u0BankCntlParm;
    }
    
    public U0idis1BankCntlParm(U0idis1BankCntlParm that) {
        this.u0idis1SysUpdTs = that.u0idis1SysUpdTs;
        this.du0u0BankCntlParm = new Du0u0BankCntlParm(that.du0u0BankCntlParm);
    }
    
    protected U0idis1BankCntlParm(byte[] bytes, int offset) {
        setBytes(bytes, offset);
    }
    
    protected U0idis1BankCntlParm(byte[] bytes) {
        this(bytes, 0);
    }
    
    public static U0idis1BankCntlParm fromBytes(byte[] bytes, int offset) {
        return new U0idis1BankCntlParm(bytes, offset);
    }
    
    public static U0idis1BankCntlParm fromBytes(byte[] bytes) {
        return fromBytes(bytes, 0);
    }
    
    public static U0idis1BankCntlParm fromBytes(String bytes) {
        return fromBytes(bytes.getBytes());
    }
    
    public String getU0idis1SysUpdTs() {
        return this.u0idis1SysUpdTs;
    }
    
    public void setU0idis1SysUpdTs(String u0idis1SysUpdTs) {
        this.u0idis1SysUpdTs = u0idis1SysUpdTs;
    }
    
    public Du0u0BankCntlParm getDu0u0BankCntlParm() {
        return this.du0u0BankCntlParm;
    }
    
    public void setDu0u0BankCntlParm(Du0u0BankCntlParm du0u0BankCntlParm) {
        this.du0u0BankCntlParm = du0u0BankCntlParm;
    }
    
    
    public String toString() {
        StringBuilder s = new StringBuilder();
        s.append("{ u0idis1SysUpdTs="");
        s.append(getU0idis1SysUpdTs());
        s.append(""");
        s.append(", du0u0BankCntlParm="");
        s.append(getDu0u0BankCntlParm());
        s.append(""");
        s.append("}");
        return s.toString();
    }
    
    public boolean equals(U0idis1BankCntlParm that) {
        return this.u0idis1SysUpdTs.equals(that.u0idis1SysUpdTs) &&
            this.du0u0BankCntlParm.equals(that.du0u0BankCntlParm);
    }
    
    @Override
    public boolean equals(Object that) {
        return (that instanceof U0idis1BankCntlParm) && this.equals((U0idis1BankCntlParm)that);
    }
    
    @Override
    public int hashCode() {
        return u0idis1SysUpdTs.hashCode() ^
            Integer.rotateLeft(du0u0BankCntlParm.hashCode(), 1);
    }
    
    @Override
    public int compareTo(U0idis1BankCntlParm that) {
        int c = 0;
        c = this.u0idis1SysUpdTs.compareTo(that.u0idis1SysUpdTs);
        if ( c != 0 ) return c;
        c = this.du0u0BankCntlParm.compareTo(that.du0u0BankCntlParm);
        return c;
    }
    
    // Start of COBOL-compatible binary serialization metadata
    private static CobolDatatypeFactory factory = new CobolDatatypeFactory();
    static {
        factory.setStringTrimDefault(true);
        factory.setStringEncoding("IBM-1047");
    }
    
    private static final StringField U0IDIS1SYSUPDTS = factory.getStringField(26);
    private static final ByteArrayField DU0U0BANKCNTLPARM = factory.getByteArrayField(Du0u0BankCntlParm.SIZE);
    public static final int SIZE = factory.getOffset();
    // End of COBOL-compatible binary serialization metadata
    
    public byte[] getBytes(byte[] bytes, int offset) {
        U0IDIS1SYSUPDTS.putString(u0idis1SysUpdTs, bytes, offset);
        du0u0BankCntlParm.getBytes(bytes, DU0U0BANKCNTLPARM.getOffset() + offset);
        return bytes;
    }
    
    public final byte[] getBytes(byte[] bytes) {
        return getBytes(bytes, 0);
    }
    
    public final byte[] getBytes() {
        return getBytes(new byte[numBytes()]);
    }
    
    public final String toByteString() {
        return new String(getBytes());
    }
    
    public void setBytes(byte[] byte

In [26]:
#Not like OO style 
#physical data access layer is embedding into the business function.  Don't see decoupling of virtual data access layer and physical data access layer, 

java = """
public class Curd004k implements Comparable<Curd004k> {
    public Curd004k() {
    }

    public Curd004k(Curd004k that) {
    }

    protected Curd004k(byte[] bytes, int offset) {
        setBytes(bytes, offset);
    }

    protected Curd004k(byte[] bytes) {
        this(bytes, 0);
    }

    public static Curd004k fromBytes(byte[] bytes, int offset) {
        return new Curd004k(bytes, offset);
    }

    public static Curd004k fromBytes(byte[] bytes) {
        return fromBytes(bytes, 0);
    }

    public static Curd004k fromBytes(String bytes) {
        try {
            return fromBytes(bytes.getBytes(factory.getStringEncoding()));
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }


    public void reset() {
    }


    public static Filler7 r100ReadCifepiRtn(ZFile input) {
        try {
            Filler7 filler7 = null;
            byte[] bytes = new byte[input.getLrecl()];
            if (input.read(bytes) == -1) return null;
            System.out.println("********Initial Read**********");
            for (byte b : bytes) {
                String str = String.format("%02X", b);
                System.out.print(str);
            }
            System.out.println();
            filler7 = new Filler7(bytes);
            return filler7;

        } catch (ZFileException e) {
            throw new RuntimeException(e);
        }
    }


    public static void programCurd004kFirstSentences() throws ZFileException {
        CntWorkArea cntWorkArea = new CntWorkArea();
        Filler7 filler7 = new Filler7();

        InAndOut<ZFile, ZFile> files = filler7.a100InitRtn();
        cntWorkArea.a200MainRtn(files);
        cntWorkArea.a300FinalRtn(files);
    }


    public static void main(String[] args) throws ZFileException {
        programCurd004kFirstSentences();
    }
"""

In [27]:
prompt = f"""[INST]<<SYS>>
java: `{java}`
<</SYS>>
please rewrite JAVA code to more OO Style.
-generate the java only.
-dont show the Notes.
-dont describe your steps.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
please rewrite JAVA code to more OO Style.\
-generate the java only.\
-dont show the Notes.\
-dont describe your steps.\
-end with ```.
java: `{java}`
Answer:```java"""

answer = model.invoke(prompt)
display(Code(answer,language="java"))

public class Curd004k implements Comparable<Curd004k> {
    public Curd004k() {
    }

    public Curd004k(Curd004k that) {
    }

    protected Curd004k(byte[] bytes, int offset) {
        setBytes(bytes, offset);
    }

    protected Curd004k(byte[] bytes) {
        this(bytes, 0);
    }

    public static Curd004k fromBytes(byte[] bytes, int offset) {
        return new Curd004k(bytes, offset);
    }

    public static Curd004k fromBytes(byte[] bytes) {
        return fromBytes(bytes, 0);
    }

    public static Curd004k fromBytes(String bytes) {
        try {
            return fromBytes(bytes.getBytes(factory.getStringEncoding()));
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }


    public void reset() {
    }


    public static Filler7 r100ReadCifepiRtn(ZFile input) {
        try {
            Filler7 filler7 = null;
            byte[] bytes = new byte[input.getLrecl()];
            if (input.read(bytes) == -1) return null;
            filler7 = new Filler7(bytes);
            return filler7;

        } catch (ZFileException e) {
            throw new RuntimeException(e);
        }
    }


    public static void programCurd004kFirstSentences() throws ZFileException {
        CntWorkArea cntWorkArea = new CntWorkArea();
        Filler7 filler7 = new Filler7();

        InAndOut<ZFile, ZFile> files = filler7.a100InitRtn();
        cntWorkArea.a200MainRtn(files);
        cntWorkArea.a300FinalRtn(files);
    }


    public static void main(String[] args) throws ZFileException {
        programCurd004kFirstSentences();
    }
}
```

In [28]:
#Not like OO style 
#any consideration of deployment for online Java service in java application server such as Liberty on z/OS. 
#for example, especially for online programs, should have separate file access service to open file rather than embeding into business service; 
#should have common connection pooling for JDBC connection rather than embeding into the business service.

java = """
    public static void b020MainProcess(long waRecRead, WwVariables wwVariables) {
    U0idis1BankCntlParm u0idis1BankCntlParm = new U0idis1BankCntlParm();
    WeErrorVariables weErrorVariables = new WeErrorVariables();

    weErrorVariables.setWeParagraphName("B020-MAIN-PROCESS");

    while (wwVariables.x020ReadInputBcftbi(u0idis1BankCntlParm, waRecRead)) {
        try {
            String sql = "INSERT INTO DU0U0_BANK_CNTL_PARM(CTRY_CDE, BANK_ID, PARM_ID, PARM_QUAL, BUS_DATE, PARM)values(?, ?, ?, ?, ?, ?)";
            String url = "jdbc:db2://192.168.7.253:9446/ZOS10101";
            Connection JdbcConnection = DriverManager.
                        getConnection(url, "TEAM##", "PASSWORD");
            PreparedStatement ps = JdbcConnection.prepareStatement(sql);
            ps.setString(1, u0idis1BankCntlParm.getDu0u0BankCntlParm().getCtryCde());
            ps.setString(2, u0idis1BankCntlParm.getDu0u0BankCntlParm().getBankId());
            ps.setString(3, u0idis1BankCntlParm.getDu0u0BankCntlParm().getParmId());
            ps.setString(4, u0idis1BankCntlParm.getDu0u0BankCntlParm().getParmQual());
            ps.setString(5, u0idis1BankCntlParm.getDu0u0BankCntlParm().getBusDate());
            ps.setString(6, u0idis1BankCntlParm.getDu0u0BankCntlParm().getParm());
            ps.executeUpdate();
            ps.close();
        }
        catch(SQLException exception) {
            weErrorVariables.setWeMsg04SqlCde(exception.getErrorCode());
            System.out.println(weErrorVariables.getWeSysoutMsgWe04());
            System.out.println(weErrorVariables.getWeSysoutMsgWa01());
            V0ibbct1.b030Termination(waRecRead, wwVariables);
        }
    }
}
"""

In [29]:
prompt = f"""[INST]<<SYS>>
java: `{java}`
<</SYS>>
please rewrite JAVA code to more Object Oriented Style.
-should have common connection pooling for JDBC connection rather than embeding into the business service.
-generate the java only.
-aware of indentation as \\t.
-dont show the Notes.
-dont describe your steps.
-end with ```.
[/INST]java:```"""

prompt = f"""Question:
please rewrite JAVA code to more Object Oriented Style.\
-should have common connection pooling for JDBC connection rather than embeding into the business service.\
-generate the java only.\
-aware of indentation as \\t.\
-dont show the Notes.\
-dont describe your steps.\
-end with ```.
java: `{java}`
Answer:```java"""

answer = model.invoke(prompt)
display(Code(answer,language="java"))

public class BankCntlParm {
    private String ctryCde;
    private String bankId;
    private String parmId;
    private String parmQual;
    private String busDate;
    private String parm;

    public String getCtryCde() {
        return ctryCde;
    }

    public void setCtryCde(String ctryCde) {
        this.ctryCde = ctryCde;
    }

    public String getBankId() {
        return bankId;
    }

    public void setBankId(String bankId) {
        this.bankId = bankId;
    }

    public String getParmId() {
        return parmId;
    }

    public void setParmId(String parmId) {
        this.parmId = parmId;
    }

    public String getParmQual() {
        return parmQual;
    }

    public void setParmQual(String parmQual) {
        this.parmQual = parmQual;
    }

    public String getBusDate() {
        return busDate;
    }

    public void setBusDate(String busDate) {
        this.busDate = busDate;
    }

    public String getParm() {
        return parm;
    }

    public void setParm(String parm) {
        this.parm = parm;
    }
}

public class BankCntlParmDAO {
    public void insert(BankCntlParm bankCntlParm) {
        String sql = "INSERT INTO DU0U0_BANK_CNTL_PARM(CTRY_CDE, BANK_ID, PARM_ID, PARM_QUAL, BUS_DATE, PARM)values(?, ?, ?, ?, ?, ?)";
        String url = "jdbc:db2://192.168.7.253:9446/ZOS10101";
        Connection JdbcConnection = DriverManager.
                    getConnection(url, "TEAM##", "PASSWORD");
        PreparedStatement ps = JdbcConnection.prepareStatement(sql);
        ps.setString(1, bankCntlParm.getCtryCde());
        ps.setString(2, bankCntlParm.getBankId());
        ps.setString(3, bankCntlParm.getParmId());
        ps.setString(4, bankCntlParm.getParmQual());
        ps.setString(5, bankCntlParm.getBusDate());
        ps.setString(6, bankCntlParm.getParm());
        ps.executeUpdate();
        ps.close();
    }
}

public class BankCntlParmService {
    private BankCntlParmDAO bankCntlParmDAO;

    public void setBankCntlParmDAO(BankCntlParmDAO bankCntlParmDAO) {
        this.bankCntlParmDAO = bankCntlParmDAO;
    }

    public void insert(BankCntlParm bankCntlParm) {
        bankCntlParmDAO.insert(bankCntlParm);
    }
}

public class BankCntlParmMain {
    public static void main(String[] args) {
        BankCntlParmDAO bankCntlParmDAO = new BankCntlParmDAO();
        BankCntlParmService bankCntlParmService = new BankCntlParmService();
        bankCntlParmService.setBankCntlParmDAO(bankCntlParmDAO);

        BankCntlParm bankCntlParm = new BankCntlParm();
        bankCntlParm.setCtryCde("CTRYCDE");
        bankCntlParm.setBankId("BANKID");
        bankCntlParm.setParmId("PARMID");
        bankCntlParm.setParmQual("PARMQUAL");
        bankCntlParm.setBusDate("BUSDATE");
        bankCntlParm.setParm("PARM");

        bankCntlParmService.insert(bankCntlParm);
    }
}
```

In [30]:
#read the java folder
#for each file
#convert it to java DDD style.
#I got few pair of sample that good for few shot learning
#how to get align

import os

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.java'):
                file_path = os.path.join(root, file)
                # print(file_path)
                file_paths.append(file_path)
    return file_paths

# Example usage
folder_path = 'wcaout/v0ibbctl.java'
uploaded_files = get_file_paths(folder_path)
print(uploaded_files)
print(len(uploaded_files))


['wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/WeErrorVariables.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/V0ibbct1.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/WwVariables.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/WcConstants.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/bcftbi.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/implementation/U0idis1BankCntlParm.java', 'wcaout/v0ibbctl.java/com/ibm/wcaz/datamodel/Du0u0BankCntlParm.java']
7


In [31]:
def generate_better_java(java):
    prompt = f"""[INST]you are java specialist, please help rewrite the java provided to better OO and DDD style.
    -employ object oriented practice.
    -employ domain driven design practice.
    -end the generation with ```.
<<SYS>>
java: `{java}`
<</SYS>>
[/INST]rewrited java: ```"""
    prompt = f"""Question:
    you are java specialist, please help rewrite the java provided to better OO and DDD style.\
    -employ object oriented practice.\
    -employ domain driven design practice.\
    -end the generation with ```.
java: `{java}`
Answer:rewrited java: ```java"""
    # print(prompt)
    answer = model.invoke(prompt)
    return answer

In [32]:
from IPython.display import display, Code, Markdown

def extract_best_practice(from_java_file,to_java_file):
    from_java = open(from_java_file,"r").read()
    to_java = open(to_java_file,"r").read()

    prompt = f"""[INST]you are java expert, that task to review a conversion of java file base on best practice.
    -please digest the changes, and only identify the best practices applied in the to_java file and not in from_java.
    -for each practice, elaborate the operation, the problem it try to solve, and the resolution.
    -in most case, to_java is more prefereable than from_java, hence only extract best practice that shown in to_java.
    -ignore the common part.
    -show the example if good example in from_java, to_java provided in comparision.
    -end with <eof>
    <<SYS>>
    from_java: `{from_java}`
    to_java: `{to_java}`
    <</SYS>>
    [/INST]best practices in markdown:"""

    prompt = f"""Question:
    you are java expert, that task to review a conversion of java file base on best practice.\
    -please digest the changes, and only identify the best practices applied in the to_java file and not in from_java.\
    -for each practice, elaborate the operation, the problem it try to solve, and the resolution.\
    -in most case, to_java is more prefereable than from_java, hence only extract best practice that shown in to_java.\
    -ignore the common part.\
    -show the example if good example in from_java, to_java provided in comparision.\
    -end with <eof>
    from_java: `{from_java}`
    to_java: `{to_java}`
    Answer:best practices in markdown:"""

    model = LangChainInterface(
            model_id="ibm/granite-20b-code-instruct-v1",
            # model_id="mistralai/mixtral-8x7b-instruct-v0-1",
            client=Client(credentials=Credentials.from_env()),
            parameters=TextGenerationParameters(
                decoding_method=DecodingMethod.GREEDY,
                max_new_tokens=3000,
                min_new_tokens=1,
                # temperature=0.05,
                top_k=50,
                top_p=1,
                stop_sequences=['<eof>'],
            )
        )

    answer = model.invoke(prompt)
    display(Markdown(answer))
    return answer

import os

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.java'):
                file_path = os.path.join(root, file)
                # print(file_path)
                file_path = file_path.replace(folder_path,"")
                file_paths.append(file_path)
    return file_paths

def consolidate_best_practice(best_practices):
    prompt = f"""[INST]you are java expert, please consolidate the best practices provided.
    -dont deplicate.
    -end with <eof>
    <<SYS>>
    best practices: `{best_practices}`
    <</SYS>>
    [/INST]consolidate best practices in markdown:"""

    prompt = f"""Question:
    you are java expert, please consolidate the best practices provided.\
    -dont deplicate.\
    -end with <eof>\
    best practices: `{best_practices}`
    Answer:consolidate best practices in markdown:"""

    model = LangChainInterface(
            model_id="ibm/granite-20b-code-instruct-v1",
            # model_id="mistralai/mixtral-8x7b-instruct-v0-1",
            client=Client(credentials=Credentials.from_env()),
            parameters=TextGenerationParameters(
                decoding_method=DecodingMethod.GREEDY,
                max_new_tokens=3000,
                min_new_tokens=1,
                # temperature=0.05,
                top_k=50,
                top_p=1,
                stop_sequences=['<eof>'],
            )
        )

    answer = keep_generate_till(model,prompt,"<eof>")
    display(Markdown(answer))
    return answer

# Example usage
from_folder_path = 'wcaout/v0ibbctl.java'
to_folder_path = "wcaout/hsbc-cobol-java-gen  revised by JAVA SME-2/hsbc-cobol-java-gen/src/main/java"
uploaded_files = get_file_paths(from_folder_path)
print(uploaded_files)
print(len(uploaded_files))

best_practices = []

for file in uploaded_files:
    from_java_file = f"{from_folder_path}{file}"
    to_java_file = f"{to_folder_path}{file}"
    if os.path.exists(from_java_file) and os.path.exists(to_java_file):
        practice = extract_best_practice(from_java_file,to_java_file)
        print("file name:"+file)
        best_practices.append(practice)

['/com/ibm/wcaz/implementation/WeErrorVariables.java', '/com/ibm/wcaz/implementation/V0ibbct1.java', '/com/ibm/wcaz/implementation/WwVariables.java', '/com/ibm/wcaz/implementation/WcConstants.java', '/com/ibm/wcaz/implementation/bcftbi.java', '/com/ibm/wcaz/implementation/U0idis1BankCntlParm.java', '/com/ibm/wcaz/datamodel/Du0u0BankCntlParm.java']
7




# Best Practices

## Use of `final` keyword

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `weParagraphName`, `weMsg04SqlCde`, `weMsg02StatCde`, `weMsg03StatCde`, `weMsg101TotCnt`, and `weMsg01StatCde` variables are declared as `final`. This means that they cannot be changed after they are initialized.

```java
public class WeErrorVariables {
    private final String weParagraphName = "";
    private final long weMsg04SqlCde = 0;
    private final String weMsg02StatCde = "";
    private final String weMsg03StatCde = "";
    private final int weMsg101TotCnt = 0;
    private final String weMsg01StatCde = "";
}
```

## Use of `equals` and `hashCode` methods

The `equals` and `hashCode` methods are used to compare objects for equality and to generate a hash code for the object.

In the `WeErrorVariables` class, the `equals` method compares two `WeErrorVariables` objects for equality by comparing the values of their `weParagraphName`, `weMsg04SqlCde`, `weMsg02StatCde`, `weMsg03StatCde`, `weMsg101TotCnt`, and `weMsg01StatCde` fields. The `hashCode` method generates a hash code by combining the hash codes of the `weParagraphName`, `weMsg04SqlCde`, `weMsg02StatCde`, `weMsg03StatCde`, `weMsg101TotCnt`, and `weMsg01StatCde` fields.

```java
public class WeErrorVariables {
    // ...
    
    @Override
    public boolean equals(Object that) {
        return (that instanceof WeErrorVariables) && this.equals((WeErrorVariables)that);
    }
    
    @Override
    public int hashCode() {
        return weParagraphName.hashCode() ^
            Integer.rotateLeft(Long.hashCode(weMsg04SqlCde), 1) ^
            Integer.rotateLeft(weMsg02StatCde.hashCode(), 2) ^
            Integer.rotateLeft(weMsg03StatCde.hashCode(), 3) ^
            Integer.rotateLeft(Integer.hashCode(weMsg101TotCnt), 4) ^
            Integer.rotateLeft(weMsg01StatCde.hashCode(), 5);
    }
}
```

## Use of `compareTo` method

The `compareTo` method is used to compare two objects of the same type.

In the `WeErrorVariables` class, the `compareTo` method compares two `WeErrorVariables` objects by comparing the values of their `weParagraphName`, `weMsg04SqlCde`, `weMsg02StatCde`, `weMsg03StatCde`, `weMsg101TotCnt`, and `weMsg01StatCde` fields.

```java
public class WeErrorVariables {
    // ...
    
    @Override
    public int compareTo(WeErrorVariables that) {
        int c = 0;
        c = this.weParagraphName.compareTo(that.weParagraphName);
        if ( c != 0 ) return c;
        c = Long.compare(this.weMsg04SqlCde, that.weMsg04SqlCde);
        if ( c != 0 ) return c;
        c = this.weMsg02StatCde.compareTo(that.weMsg02StatCde);
        if ( c != 0 ) return c;
        c = this.weMsg03StatCde.compareTo(that.weMsg03StatCde);
        if ( c != 0 ) return c;
        c = Integer.compare(this.weMsg101TotCnt, that.weMsg101TotCnt);
        if ( c != 0 ) return c;
        c = this.weMsg01StatCde.compareTo(that.weMsg01StatCde);
        return c;
    }
}
```

## Use of `getBytes` and `setBytes` methods

The `getBytes` and `setBytes` methods are used to convert the object to and from a byte array.

In the `WeErrorVariables` class, the `getBytes` method converts the object to a byte array by using the COBOL-compatible binary serialization metadata defined at the top of the class. The `setBytes` method converts the byte array to an object by using the COBOL-compatible binary serialization metadata.

```java
public class WeErrorVariables {
    // ...
    
    public byte[] getBytes(byte[] bytes, int offset) {
        WEPARAGRAPHNAME.putString(weParagraphName, bytes, offset);
        WEMSG04SQLCDE.putLong(weMsg04SqlCde, bytes, offset);
        WEMSG02STATCDE.putString(weMsg02StatCde, bytes, offset);
        WEMSG03STATCDE.putString(weMsg03StatCde, bytes, offset);
        WEMSG101TOTCNT.putInt(weMsg101TotCnt, bytes, offset);
        WEMSG01STATCDE.putString(weMsg01StatCde, bytes, offset);
        return bytes;
    }
    
    public final byte[] getBytes(byte[] bytes) {
        return getBytes(bytes, 0);
    }
    
    public final byte[] getBytes() {
        return getBytes(new byte[numBytes()]);
    }
    
    public final String toByteString() {
        try {
            return new String(getBytes(), factory.getStringEncoding());
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }
    
    public void setBytes(byte[] bytes, int offset) {
        if (bytes.length < SIZE + offset) {
            byte[] newBytes = Arrays.copyOf(bytes, SIZE + offset);
            Arrays.fill(newBytes, bytes.length, SIZE + offset, (byte)0x40 /*default EBCDIC space character*/);
            bytes = newBytes;
        }
        weParagraphName = WEPARAGRAPHNAME.getString(bytes, offset);
        weMsg04SqlCde = WEMSG04SQLCDE.getLong(bytes, offset);
        weMsg02StatCde = WEMSG02STATCDE.getString(bytes, offset);
        weMsg03StatCde = WEMSG03STATCDE.getString(bytes, offset);
        weMsg101TotCnt = WEMSG101TOTCNT.getInt(bytes, offset);
        weMsg01StatCde = WEMSG01STATCDE.getString(bytes, offset);
    }
    
    
    public final void setBytes(byte[] bytes) {
        setBytes(bytes, 0);
    }
    
    public final void setBytes(String bytes) {
        try {
            setBytes(bytes.getBytes(factory.getStringEncoding()));
        } catch (UnsupportedEncodingException e) {
            throw new RuntimeException(e);
        }
    }
    
    public int numBytes() {
        return SIZE;
    }
}
```

## Use of `final` keyword in `CobolDatatypeFactory`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `factory` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final CobolDatatypeFactory factory = new CobolDatatypeFactory();
    static {
        factory.setStringTrimDefault(true);
        factory.setStringEncoding("IBM-1047");
    }
}
```

## Use of `final` keyword in `StringField`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `WEPARAGRAPHNAME`, `WEMSG02STATCDE`, `WEMSG03STATCDE`, and `WEMSG01STATCDE` variables are declared as `final`. This means that they cannot be changed after they are initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final StringField WEPARAGRAPHNAME = factory.getStringField(30);
    private static final StringField WEMSG02STATCDE = factory.getStringField(2);
    private static final StringField WEMSG03STATCDE = factory.getStringField(2);
    private static final StringField WEMSG01STATCDE = factory.getStringField(2);
}
```

## Use of `final` keyword in `ExternalDecimalAsLongField`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `WEMSG04SQLCDE` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final ExternalDecimalAsLongField WEMSG04SQLCDE = factory.getExternalDecimalAsLongField(10, true);
}
```

## Use of `final` keyword in `ExternalDecimalAsIntField`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `WEMSG101TOTCNT` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final ExternalDecimalAsIntField WEMSG101TOTCNT = factory.getExternalDecimalAsIntField(5, true);
}
```

## Use of `final` keyword in `SIZE`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `SIZE` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    public static final int SIZE = factory.getOffset();
}
```

## Use of `final` keyword in `factory`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `factory` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final CobolDatatypeFactory factory = new CobolDatatypeFactory();
    static {
        factory.setStringTrimDefault(true);
        factory.setStringEncoding("IBM-1047");
    }
}
```

## Use of `final` keyword in `factory`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `factory` variable is declared as `final`. This means that it cannot be changed after it is initialized.

```java
public class WeErrorVariables {
    // ...
    
    private static final CobolDatatypeFactory factory = new CobolDatatypeFactory();
    static {
        factory.setStringTrimDefault(true);
        factory.setStringEncoding("IBM-1047");
    }
}
```

## Use of `final` keyword in `factory`

The `final` keyword is used to indicate that a variable is not intended to be changed. It is used to prevent accidental changes to a variable, and to ensure that the variable is initialized before it is used.

In the `WeErrorVariables` class, the `factory` variable is declared as `final`. This means that it cannot

file name:/com/ibm/wcaz/implementation/WeErrorVariables.java


In [6]:
def keep_generate_till(model,prompt,endstring):
    lastanswer = ""
    while not lastanswer.endswith(endstring):
        answer = model.invoke(f"{prompt}{lastanswer}")
        lastanswer += answer
    return lastanswer

practice = consolidate_best_practice(best_practices)
display(Markdown(practice))



Java Best Practices
------------------

- **Avoid Duplication**
	- Avoid duplicating code or logic in your program.
- **End with <eof>



Java Best Practices
------------------

- **Avoid Duplication**
	- Avoid duplicating code or logic in your program.
- **End with <eof>

In [ ]:
def split_string(string, chunk_size):
    return [string[i:i+chunk_size] for i in range(0, len(string), chunk_size)]

In [ ]:
import difflib
from IPython.display import HTML

for file in uploaded_files:
    java = open(file,"r").read()
    javaparts = [java]
    # javaparts = [split_string(java,2000)]
    for javapart in javaparts:
        betterjava = generate_better_java(javapart)
        display(Code(betterjava,language="java"))

    # diffstring = difflib.HtmlDiff().make_file(java.splitlines(), betterjava.splitlines())
    # display(HTML(diffstring))

In [ ]:
import json
from anytree import Node, RenderTree

def serialize_tree(root_node):
    serialized_data = []
    for pre, _, node in RenderTree(root_node):
        serialized_data.append({
            "name": node.name,
            "parent": node.parent.name if node.parent else None,
            "children": [child.name for child in node.children]
        })
    return serialized_data

# Create nodes
root = Node("class ABC{")
node_b = Node("int DEF;", parent=root)
node_c = Node("}", parent=root)
node_d = Node("void bar(){}", parent=node_b)

# Serialize the tree
serialized_tree = serialize_tree(root)

# Convert to JSON
json_data = json.dumps(serialized_tree, indent=4)

# Print the JSON data
# print(json_data)
print(root.name)
for child in root.children:
    print("\t"+child.name)
    for childchild in child.children:
        print("\t\t"+childchild.name)

In [ ]:

prompt = f"""[INST]you are java specialist, please help rewrite the java provided to better OO and DDD style.
    -employ object oriented practice.
    -employ domain driven design practice.
    -end the generation with ```.
    <<SYS>>
    java: `{java}`
    <</SYS>>
    [/INST]rewrited java: ```"""
# print(prompt)
answer = ""
for response in model.generate_text(prompt):
    answer += response
print(Code(answer,language="java"))